In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dwave.inspector
import QA_DwaveSampler_AutoEmbedding as qda
import pandas as pd
import numpy as np
import timeit
import matplotlib.pyplot as plt
import statistics

##### autoembeddingcompositeとembeddingcompositeの違いは，自動でマイナー埋め込みをする必要があるか判断してくれるか（autoembeddingcomposite），自動でマイナー埋め込みを常にしてくれるか（embeddingcomposite）．現在autoembeddingcompositeを使っているが，調べるとembeddingcompositeを使っている例の方が多い．autoembeddingcompositeはまずはそのままのbqmで試して，BinaryQuadraticModelStructureErrorというサンプラーが扱えない形のbqmが入った際に出るエラーが発令されたら，マイナー埋め込みをするらしい．大きな問題だと埋め込みに時間がかかるので，しなくて済むならしないでおきたい．そのニーズに答えるためのツールなのか．

マイナー埋め込みは手動と自動どちらも可能．自動で行うというのは，Oceanソフトウェア・ツールを使って行うことを指す．

autoのほうが速いかと思って試してみたが一回目の試行ではautonのほうが負けていたので途中でやめた

In [ ]:
bit, num, n, num_reads = 20, 0, 100, 100
path = '../input/ost{}_{}.csv'.format(bit, num)
df = pd.read_csv(path, sep=',')
bqm = qda.make_Hamiltonian(df)

In [ ]:
from dimod.reference.samplers import ExactSolver
solver = ExactSolver()
response = solver.sample(bqm)
for datum in response.data(['sample', 'energy']):     
    if datum.energy==0.0:
        datum.sample.values()

In [ ]:
bit, num, n, num_reads = 20, 0, 100, 100
path = '../input/ost{}_{}.csv'.format(bit, num)
df = pd.read_csv(path, sep=',')
bqm = qda.make_Hamiltonian(df)
#QDA = qda.QA_DSampler_AEmbedding(df, bqm, n, num_reads, chain_strength)

#繰り返さない！
dw_sampler = DWaveSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09",
        retry_interval=5
    )
# インスタンス作成
qa_sampler = AutoEmbeddingComposite(dw_sampler)

res = qa_sampler.sample(bqm, chain_strength = chain_strength, chain_break_fraction=True, num_reads=num_reads)
# Inspect
dwave.inspector.show(response)

### chain_strength = partial(uniform_torque_compensation, prefactor=2) ???

- chain_strength (float, optional) – The (relative) chain strength to use in the embedding. By default a chain strength of 1.5sqrt(N) where N is the size of the largest clique, as returned by largest_clique_size.

- chain_strength (float/mapping/callable, optional, default=1.0) Magnitude of the quadratic bias (in SPIN-space) applied between variables to create chains. The energy penalty of chain breaks is 2 * chain_strength.mappingが渡されたらchain特有の強さが適用される. If a callable is passed, it will be called on chain_strength(bqm, embedding) and should return a float or mapping, to be interpreted as above. By default, chain_strength is scaled to the problem. デフォルトでも，考えられた値となっているようだ．autoembedding_compositeではこのように考えられた値だが，embedding_compositeではnoneとなっているようだ．一方両者ともにdefaul=1.0と書いてある

→returning_embeddin=Trueで確認したらchain_strengthとかわかりそう

- return_embedding (bool, optional) – If True, the embedding, chain strength, chain break method and embedding parameters are added to dimod. SampleSet.info of the returned sample set. The default behaviour is defined by return_embedding_default, which itself defaults to False.

In [ ]:
#autoを使わないと精度は変わるのか

from functools import partial
>>> from dwave.system import EmbeddingComposite, DWaveSampler
>>> from dwave.embedding.chain_strength import uniform_torque_compensation
...
>>> Q = {(0,0): 1, (1,1): 1, (2,3): 2, (1,2): -2, (0,3): -2}
>>> sampler = EmbeddingComposite(DWaveSampler())
>>> # partial() can be used when the BQM or embedding is not accessible
>>> chain_strength = partial(uniform_torque_compensation, prefactor=2)
>>> sampleset = sampler.sample_qubo(Q, chain_strength=chain_strength, return_embedding=True)
>>> sampleset.info['embedding_context']['chain_strength']

- [Ocean SDK](https://docs.ocean.dwavesys.com/en/latest/)は、D-Wave量子コンピューターをプログラミングするためのオープンソースのPythonツールのセット

In [1]:
import networkx as nx
 
graph_nodes = 16
G = nx.random_geometric_graph(n=graph_nodes, radius=.5, dim=2)

In [2]:
from collections import defaultdict
from itertools import combinations
 
gamma = 60
 
Q = defaultdict(int)
 
# Fill in Q matrix
for u, v in G.edges:
    Q[(u,u)] += 1
    Q[(v,v)] += 1
    Q[(u,v)] += -2

for i in G.nodes:
    Q[(i,i)] += gamma*(1-len(G.nodes))

for i, j in combinations(G.nodes, 2):
    Q[(i,j)] += 2*gamma

In [3]:
print("Maximum element is {:.2f} and minimum is {:.2f}.".format(max(Q.values()), min(Q.values())))    # doctest: +SKIP

Maximum element is 120.00 and minimum is -897.00.


In [4]:
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite
 
# Import the problem inspector to begin data capture
import dwave.inspector
 
sampler = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))
 
num_reads = 1000
sampleset = sampler.sample_qubo(Q, num_reads=num_reads)

In [5]:
print("Number of nodes in one set is {}, in the other, {}. \nEnergy is {}.".format(
    sum(sampleset.first.sample.values()),
    graph_nodes - sum(sampleset.first.sample.values()),
    sampleset.first.energy))      # doctest: +SKIP

Number of nodes in one set is 8, in the other, 8. 
Energy is -3809.0.


In [6]:
print("Percentage of samples with high rates of breaks is {}.".format(
    np.count_nonzero(sampleset.record.chain_break_fraction > 0.33)/num_reads*100))    # doctest: +SKIP

Percentage of samples with high rates of breaks is 6.6000000000000005.


In [7]:
dwave.inspector.show(sampleset)

'http://127.0.0.1:18000/?problemId=19fa8c91-64f6-4d07-a85f-8dbdacd7e6b0'

In [8]:
sampleset = sampler.sample_qubo(Q, num_reads=num_reads, chain_strength=1000)

In [9]:
print("Number of nodes in one set is {}, in the other, {}. \nEnergy is {}.".format(
...        sum(sampleset.first.sample.values()),
...        graph_nodes - sum(sampleset.first.sample.values()),
...        sampleset.first.energy))    # doctest: +SKIP

Number of nodes in one set is 8, in the other, 8. 
Energy is -3814.0.


In [10]:
print("Percentage of samples with high rates of breaks is {}.".format(
...        np.count_nonzero(sampleset.record.chain_break_fraction > 0.33)/num_reads*100))    # doctest: +SKIP

Percentage of samples with high rates of breaks is 0.0.


In [11]:
dwave.inspector.show(sampleset)

'http://127.0.0.1:18000/?problemId=54c66c0a-e191-472a-822c-f8de9dc724d1'

In [13]:
!python graph_partitioning.py

python: can't open file 'graph_partitioning.py': [Errno 2] No such file or directory
